# LMP Pricing

#### The Goal of this notebook is to learn about locational marginal pricing.

General steps will be:

1) Generate supply and demand side bids
2) Calculate LMP's from these bids

This notebook is mainly to be used as a heuristic tool in learning about LMP's and CAISO market operations. I'm sure anyone can use ChatGPT to generate a pricing algorithm in seconds, but that teaches us nothing .

In [3]:
import numpy as np
import pandas as pd
from typing import List, Tuple
import random

First, I want to simulate both supply and demand side bids. 

There are generators and buyers (typically load serving entities AKA LSE's) who mainly participate in the wholesale market. 

They submit day-ahead bids to procure to provide energy.

In [46]:
# both functions sort by merit order (supply) and willingness to pay (demand)

def generate_supply_bids(n_bids=5, price_range=(10, 50), quantity_range=(10, 100)):
    """
    Generate a DataFrame of supply offers (price $/MWh, quantity MW).
    Sorted ascending by price (merit order).
    """
    bids = [(random.randint(*price_range), random.randint(*quantity_range)) for _ in range(n_bids)]
    bids = sorted(bids, key=lambda x: x[0])
    return pd.DataFrame(bids, columns=['Price ($/MWh)', 'Quantity (MW)'])

def generate_demand_bids(n_bids=5, price_range=(30, 100), quantity_range=(10, 100)):
    """
    Generate a DataFrame of demand bids (price $/MWh, quantity MW).
    Sorted descending by price (highest willingness to pay first).
    """
    bids = [(random.randint(*price_range), random.randint(*quantity_range)) for _ in range(n_bids)]
    bids = sorted(bids, key=lambda x: -x[0])
    return pd.DataFrame(bids, columns=['Price ($/MWh)', 'Quantity (MW)'])


In [49]:
supply_bids = generate_supply_bids(n_bids = 20)
demand_bids = generate_demand_bids(n_bids = 20)

,Price ($/MWh),Quantity (MW)
0,12,37
1,12,12
2,12,20
3,13,35
4,16,71
5,17,42
6,17,50
7,23,37
8,26,46
9,31,50
